In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

from datasets import Dataset
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer

/home/m/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/m/.local/lib/python3.11/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [2]:
# Model from Hugging Face hub
base_model_name  = "../llama/Llama-2-7b-chat-hf"

# Fine-tuned model
new_model = "llama-2-7b-chat-arxiv"
output_dir = "./results_arxiv"

max_seq_length = 512
device_map = {"": 0}

output_dir_path = os.path.join(output_dir, "final_checkpoint")

In [10]:
from peft import AutoPeftModelForCausalLM
ckpt_dir = './results_arxiv/final_checkpoint'

model = AutoPeftModelForCausalLM.from_pretrained(
    ckpt_dir, device_map=device_map, torch_dtype=torch.bfloat16)

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.95s/it]


In [3]:
dataset = Dataset.from_csv('data/extracted_data/question_answer_dataset_from_arxiv.csv')

In [4]:
# print the first few examples of the dataset
print(dataset[:3])

{'text': ['<s>[INST] : What is the accuracy of the proposed fusion model for predicting the functional outcome of stroke patients? [/INST]: The proposed fusion model achieves an accuracy of 0.87, 0.80, and 80.45% for AUC, F1-score, and accuracy, respectively, outperforming existing models that consolidate both imaging and structured data in the medical domain. </s>', '<s>[INST] : Can diffusion-weighted MRI replace NIHSS for better generalization in predicting the functional outcome of stroke patients? [/INST]: Yes, diffusion-weighted MRI can replace NIHSS for better generalization in predicting the functional outcome of stroke patients. The addition of diffusion-weighted MRI alone can further improve the accuracy of prediction, and can also replace NIHSS for better generalization. </s>', '<s>[INST] : What are the three scientific questions that are answered in the text enclosed in double angled brackets? [/INST]:\n1. How do aspects of watermark design, such as watermark length, number 

In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

In [6]:
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True,
    use_auth_token=True
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1 

/home/m/.local/lib/python3.11/site-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.78s/it]


In [7]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [8]:
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=10,
    max_steps=100
)

In [9]:
trainer = SFTTrainer(
    model=base_model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_args,
)

trainer.train()

Map: 100%|██████████| 3/3 [00:00<00:00, 102.89 examples/s]
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,0.676000
20,0.374000
30,0.153900
40,0.045500
50,0.012500
60,0.007600
70,0.006500
80,0.006000
90,0.006100
100,0.005900


TrainOutput(global_step=100, training_loss=0.12940507013350724, metrics={'train_runtime': 92.2565, 'train_samples_per_second': 17.343, 'train_steps_per_second': 1.084, 'total_flos': 1781088856473600.0, 'train_loss': 0.12940507013350724, 'epoch': 100.0})

In [10]:
trainer.model.save_pretrained(output_dir_path)